In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

In [5]:
pd.options.display.float_format = '{:,.0f}'.format

In [6]:
train_reader = pd.read_csv('Data/Springleaf/train.csv.zip', sep=',', iterator=True, low_memory=False)
test_reader = pd.read_csv('Data/Springleaf/test.csv.zip', sep=',', iterator=True, low_memory=False)

In [7]:
train = train_reader.get_chunk(30000)

In [8]:
test = test_reader.get_chunk(10000)

In [9]:
df_train = train.copy()

In [10]:
def display_all(df):
    with pd.option_context('display.max_rows', 10000, 'display.max_columns', 10000):
        display(df)

In [11]:
display_all(df_train.head(10))

ID VAR_0001  VAR_0002  VAR_0003  VAR_0004 VAR_0005  VAR_0006  VAR_0007  \
0   2        H       224         0      4300        C         0         0   
1   4        H         7        53      4448        B         1         0   
2   5        H       116         3      3464        C         0         0   
3   7        H       240       300      3200        C         0         0   
4   8        R        72       261      2000        N         0         0   
5  14        R         4         4      4422        C         0         0   
6  16        H        60       132     40000        C         1         1   
7  20        R        13        75      3600        B         0         0   
8  21        R        17        16      2296        N         1         1   
9  22        R        24        72       450        N         0         0   

  VAR_0008 VAR_0009 VAR_0010 VAR_0011 VAR_0012  VAR_0013  VAR_0014  VAR_0015  \
0    False    False    False    False    False         0         0         0   
1    False    False    False    False    False         1         0         1   
2    False    False    False    False    False         0         0         0   
3    False    False    False    False    False         0         0         0   
4    False    False    False    False    False         0         0         0   
5    False    False    False    False    False         0         0         0   
6    False    False    False    False    False         1         1         1   
7    False    False    False    False    False         0         0         0   
8    False    False    False    False    False         1         1         1   
9    False    False    False    False    False         0         0         0   

   VAR_0016  VAR_0017  VAR_0018  VAR_0019  VAR_0020  VAR_0021  VAR_0022  \
0         1         0         0         0         0         0         0   
1         2         1         0         0         0         0         0   
2         1         0         0         0         0         0         0   
3         2         0         0         0         0         0         0   
4         1         0         0         0         0         0         0   
5         1         0         0         0         0         0         0   
6         2         0         0         0         0         0         0   
7         1         0         0         0         0         0         0   
8         1         1         0         0         0         0         0   
9         1         0         0         0         0         0         0   

   VAR_0023  VAR_0024  VAR_0025  VAR_0026  VAR_0027  VAR_0028  VAR_0029  \
0         0         0         0         0         0         0         0   
1         0         0         0         0         0         0         0   
2         0         0         0         0         0         0         0   
3         0         0         0         0         0         0         0   
4         0         0         0         0         0         0         0   
5         0         0         0         0         0         0         0   
6         0         0         0         0         0         0         0   
7         0         0         0         0         0         0         0   
8         0         0         0         0         0         0         0   
9         0         0         0         0         0         0         0   

   VAR_0030  VAR_0031  VAR_0032  VAR_0033  VAR_0034  VAR_0035  VAR_0036  \
0         0         0         0         1         0         0         0   
1         0         0         0         1         0         0         0   
2         0         0         0         1         0         0         0   
3         0         0         0         1         0         0         0   
4         0         0         0         1         0         0         0   
5         0         0         0         1         0         0         0   
6         0         0         0         2         2         2         1   
7         0         0 

Before doing any EDA, by eyeballing the data I can see that so many columns are repeated

## Data Preprocessing

### Drop empty variable 

In [11]:
# Drop VAR_0044 after noticing that it only holds empty lists
df_train.drop('VAR_0044', axis = 1, inplace=True)

In [12]:
# Extract numerical variables to run diff function
from pandas.api.types import is_string_dtype, is_numeric_dtype

def numeric(df):
    lists=[]
    for name, col in df.items():
        if is_numeric_dtype(df[name]):
            lists.append(name)
    return df[lists]

In [13]:
df_train.shape

(30000, 1933)

### 1. Drop varaibles with one level

In [14]:
unique_counts = df_train.nunique(dropna = False)

In [15]:
drop_unique = unique_counts.loc[unique_counts==1]
list(drop_unique.index)

['VAR_0207', 'VAR_0213', 'VAR_0840', 'VAR_0847', 'VAR_1428']

In [16]:
df_train.drop(list(drop_unique.index), axis=1,inplace=True)

In [17]:
df_train.shape

(30000, 1928)

### 2. Drop variables that have similar values

In [18]:
train_encoded = df_train.copy()
for col in train_encoded.columns:
    train_encoded[col] = train_encoded[col].factorize()[0] #factorize variables label encode levels in every variable

In [21]:
import pickle as pickle

In [20]:
# duplicates = {}
# for i, c1 in enumerate(train_encoded.columns):
#     for c2 in train_encoded.columns[i+1:]:
#         if c2 not in duplicates and np.all(train_encoded[c1] == train_encoded[c2]):
#             duplicates[c2] = c1

KeyboardInterrupt: 

In [167]:
# pickle.dump(duplicates, open('duplicates.p', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
duplicates = pickle.load( open( "duplicates.p", "rb" ) )

In [23]:
duplicates

{'VAR_0013': 'VAR_0006',
 'VAR_0009': 'VAR_0008',
 'VAR_0010': 'VAR_0008',
 'VAR_0011': 'VAR_0008',
 'VAR_0012': 'VAR_0008',
 'VAR_0018': 'VAR_0008',
 'VAR_0019': 'VAR_0008',
 'VAR_0020': 'VAR_0008',
 'VAR_0021': 'VAR_0008',
 'VAR_0022': 'VAR_0008',
 'VAR_0023': 'VAR_0008',
 'VAR_0024': 'VAR_0008',
 'VAR_0025': 'VAR_0008',
 'VAR_0026': 'VAR_0008',
 'VAR_0027': 'VAR_0008',
 'VAR_0028': 'VAR_0008',
 'VAR_0029': 'VAR_0008',
 'VAR_0030': 'VAR_0008',
 'VAR_0031': 'VAR_0008',
 'VAR_0032': 'VAR_0008',
 'VAR_0038': 'VAR_0008',
 'VAR_0039': 'VAR_0008',
 'VAR_0040': 'VAR_0008',
 'VAR_0041': 'VAR_0008',
 'VAR_0042': 'VAR_0008',
 'VAR_0043': 'VAR_0008',
 'VAR_0196': 'VAR_0008',
 'VAR_0197': 'VAR_0008',
 'VAR_0199': 'VAR_0008',
 'VAR_0202': 'VAR_0008',
 'VAR_0203': 'VAR_0008',
 'VAR_0215': 'VAR_0008',
 'VAR_0216': 'VAR_0008',
 'VAR_0221': 'VAR_0008',
 'VAR_0222': 'VAR_0008',
 'VAR_0223': 'VAR_0008',
 'VAR_0229': 'VAR_0008',
 'VAR_0239': 'VAR_0008',
 'VAR_0201': 'VAR_0051',
 'VAR_0238': 'VAR_0089',


In [24]:
duplicates.keys()

dict_keys(['VAR_0013', 'VAR_0009', 'VAR_0010', 'VAR_0011', 'VAR_0012', 'VAR_0018', 'VAR_0019', 'VAR_0020', 'VAR_0021', 'VAR_0022', 'VAR_0023', 'VAR_0024', 'VAR_0025', 'VAR_0026', 'VAR_0027', 'VAR_0028', 'VAR_0029', 'VAR_0030', 'VAR_0031', 'VAR_0032', 'VAR_0038', 'VAR_0039', 'VAR_0040', 'VAR_0041', 'VAR_0042', 'VAR_0043', 'VAR_0196', 'VAR_0197', 'VAR_0199', 'VAR_0202', 'VAR_0203', 'VAR_0215', 'VAR_0216', 'VAR_0221', 'VAR_0222', 'VAR_0223', 'VAR_0229', 'VAR_0239', 'VAR_0201', 'VAR_0238', 'VAR_0188', 'VAR_0189', 'VAR_0190', 'VAR_0191', 'VAR_0130', 'VAR_0181', 'VAR_0182', 'VAR_0210', 'VAR_0211', 'VAR_0228', 'VAR_0394', 'VAR_0411', 'VAR_0438', 'VAR_0446', 'VAR_0526', 'VAR_0527', 'VAR_0528', 'VAR_0529', 'VAR_0530', 'VAR_0357', 'VAR_0396', 'VAR_0398', 'VAR_0459', 'VAR_0512', 'VAR_0671', 'VAR_0672', 'VAR_1036', 'VAR_1849'])

In [25]:
df_train.drop(duplicates.keys(), axis=1, inplace=True)

In [26]:
df_train.shape

(30000, 1860)

Dropped 68 duplicate columns that had no values

### 3. Determine datatypes of features

In [28]:
display_all(df_train.head().T)

,0,1,2,3,4
ID,2,4,5,7,8
VAR_0001,H,H,H,H,R
VAR_0002,224,7,116,240,72
VAR_0003,0,53,3,300,261
VAR_0004,4300,4448,3464,3200,2000
VAR_0005,C,B,C,C,N
VAR_0006,0,1,0,0,0
VAR_0007,0,0,0,0,0
VAR_0008,False,False,False,False,False
VAR_0014,0,0,0,0,0


In [78]:
# Convert these to datetime
dates = ['VAR_0178', 'VAR_0179', 'VAR_0217', 'VAR_0073', 'VAR_0075', 'VAR_0166', 'VAR_0168', 'VAR_0169', 'VAR_0176']

for d in dates:
    df_train[d] = df_train[d].str[:7]
    df_train[d] = pd.to_datetime(df_train[d], dayfirst=True)

In [ ]:
# Convert objects into categories

In [ ]:
# What is it with the variables -1

Check if there are any fully empty variables

In [14]:
# Drop variables that contain no data
df_train.drop(['VAR_0207', 'VAR_0213', 'VAR_0840', 'VAR_0207'], axis=1, inplace=True)

In [22]:
# This looks like a date 



NameError: name 'VAR_0073' is not defined